# Programa auxiliar para generar imágenes de Carteles (con su correspondiente XML) combinando al azar las imágenes de números independientes disponibles

# Preparación del entorno:

In [1]:
#@title Cargar Librerías
import os
import os.path
from os.path import isfile, join

import random

import numpy as np

from PIL import Image as ImPIL
import csv

import xml.etree.cElementTree as ET

print ("Librerías cargadas.")

Librerías cargadas.


In [2]:
#@title Montar  Drive

from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


# Preparación de Imágenes base:

In [3]:
#@title Cargar Imágenes base a utilizar


# directorio local donde están disponibles las imágenes fuente de imágenes (están divididos por clases)
pathFuentes = '/content/gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/train, /content/gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/test' #@param {type:"string"}

if "," in pathFuentes:
  listPathFuentes = pathFuentes.split(',')
  listPathFuentes = [s.strip() for s in listPathFuentes]
else:
  listPathFuentes = [pathFuentes]


clasesFN = {}
image_extensions = ['.jpg', '.jpeg', '.png' ]

# controla si el archivo es una imagen
def esFNImagen(fn):
  _, ext = os.path.splitext(fn)
  return ext.lower() in image_extensions

for pathFuente in listPathFuentes:

  print("\n> procesando: ", pathFuente)

  # determina subdirectorios
  all_images_array = []
  all_dirs = os.listdir(pathFuente)

  # procesa cada directorio
  allFileNames = []
  for each_dir in all_dirs:
    if each_dir not in clasesFN:
      clasesFN[each_dir] = []

    # levanta la lista de imágenes de cada clase
    auxFileNames = os.listdir( pathFuente + '/' + each_dir )

    # le agrega el path completo
    for fn in auxFileNames:
      if esFNImagen(fn):
        clasesFN[each_dir].append( pathFuente + '/' + each_dir + '/' + fn )

# muestra resumen
print('\n> Clases disponibles [', len(clasesFN.keys()), ']:')
for k in clasesFN:
  print("\t", k, ":", len(clasesFN[k]))



> procesando:  /content/gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/train

> procesando:  /content/gdrive/MyDrive/IA/demoML/imagenes/NUMEROS/test

> Clases disponibles [ 10 ]:
	 0 : 30
	 4 : 30
	 5 : 30
	 6 : 30
	 8 : 30
	 1 : 30
	 9 : 30
	 7 : 30
	 2 : 30
	 3 : 30


# Generación de carteles (imágenes + XMLs):

In [7]:
#@title Generar carteles ubicando al azar Imágenes base

###

#@markdown Parámetros Generales

# cantidad de imágenes combinadas a generar
cantImagenesGenerar = 1000 #@param {type:"integer"}

#@markdown Parámetros Selección de Imágenes base:

# cantidad mínima/máxima de imágenes disponibles seleccionadas para incluir por imagen a generar combinada
cantImagenesSeleccMin = 3 #@param {type:"integer"}
cantImagenesSeleccMax = 15 #@param {type:"integer"}

cantImagenesSeleccMin = min(max(cantImagenesSeleccMin, 1), cantImagenesSeleccMax)
cantImagenesSeleccMin = min(max(cantImagenesSeleccMax, 1), 100)

permitirImagenesRepetidas = True #@param {type:"boolean"}

# indica si las imágenes elegidas se eliminan o no
# (sino puede volver a ser elegida)
eliminar_elegidas = not(permitirImagenesRepetidas)


#@markdown Parámetros Tamaño de Imágenes base:

ajustarTamanoImagenes = True #@param {type:"boolean"}

porcAjusteTamImagenMin = 0.25 #@param {type:"number"}
porcAjusteTamImagenMax = 1.25 #@param {type:"number"}

porcAjusteTamImagenMin = min(max(porcAjusteTamImagenMin, 0.1), 2.0)
porcAjusteTamImagenMax = min(max(porcAjusteTamImagenMax, porcAjusteTamImagenMin), 2.0)

aplicarMismoAjusteAltoAncho = True #@param {type:"boolean"}

#@markdown Parámetros Ubicación de Imágenes base:

ubicacionImagenesGeneradas = "Ambas" #@param [ "Mismo Renglon",  "Misma Columna", "Ambas" ]

if ubicacionImagenesGeneradas == "Mismo Renglon":
  imGeneradasDespX = True
  imGeneradasDespY = False
elif ubicacionImagenesGeneradas == "Misma Columna":
  imGeneradasDespX = False
  imGeneradasDespY = True
else: #  "Ambas"
  imGeneradasDespX = True
  imGeneradasDespY = True

distanciaImagenesGeneradasX = -1 #@param {type:"integer"}
distanciaImagenesGeneradasY = -1 #@param {type:"integer"}
#@markdown - nota: si usa distancia con un valor negativo se considera una distancia definida al azar.

if distanciaImagenesGeneradasX<0:
  distanciaImagenesGeneradasX = -1 # al AZAR
else:
  distanciaImagenesGeneradasX = min(max(distanciaImagenesGeneradasX, 1), 50)

if distanciaImagenesGeneradasY<0:
  distanciaImagenesGeneradasY = -1 # al AZAR
else:
  distanciaImagenesGeneradasY = min(max(distanciaImagenesGeneradasY, 1), 50)

# función auxiliara para determinar la distancia entre las imágenes
def determinaDistanciaImagenes(paramDistancia, tamImagen):
  if paramDistancia<0:
    return random.randint(1, max(tamImagen//2, 10))
  else:
    return paramDistancia


#@markdown Parámetros de Ajuste para XML:

# pixeles para ajustar coordenadas para XML
XMLajustePosicionesX = 10 #@param {type:"integer"}
XMLajustePosicionesY = 5 #@param {type:"integer"}


#@markdown Parámetros Archivos de Imágenes Destino:

# directorio local donde se guardarán las nuevas imágenes generadas (las pone todas juntas, luego decide si son de entrenamiento o prueba)
pathDestino = '/content/gdrive/MyDrive/IA/demoObjDet-Carteles-YOLO/Carteles/Generados' #@param {type:"string"}

# prefijo nombre imágenes a generar
prefijoImagGen = 'cartel' #@param {type:"string"}

###

# crea el directorio destino, si es necesario
if not os.path.isdir(pathDestino):
  os.makedirs(pathDestino)

# Genera la cantidad de imágenes indicadas en los parámetros
list_clases = []
clasesListDisp = list(clasesFN.keys())
auxiCantMostrarProgreso = cantImagenesGenerar//5
if "_" not in prefijoImagGen:
  prefijoImagGen = prefijoImagGen + "_"
print('\n** Comienza proceso de generación de imágenes **\n')
for i in range( cantImagenesGenerar ):

  if len(clasesListDisp)==0:
    print("-- no hay más imágenes disponibles para seleccionar.")
    break

  # define el nombre de la imagen a generar
  nombreImagComb = prefijoImagGen

  #  determina al azar la cantidad de imágenes para utilizar como base
  seleccImag_cant = random.randint(cantImagenesSeleccMin, cantImagenesSeleccMax)

  # define un tamaño inicial para poner toda la imagen
  nuevaImgAnchoTotal = 1024
  nuevaImgAltoTotal = 1024

  # genera una imagen grande para que entre todo y luego lo achica
  nuevaImg  = ImPIL.new('RGB', (nuevaImgAnchoTotal, nuevaImgAltoTotal), "WHITE")

  # inicializa variables auxiliares
  ultimaUbicacionX1 = 0
  ultimaUbicacionX2 = 0
  ultimaUbicacionY1 = 0
  ultimaUbicacionY2 = 0
  maximaUbicacionX = 0
  minimaUbicacionY = 0
  maximaUbicacionY = 0
  xml_annotation_list = []

  # toma una cantidad de imagenes para agregar
  for j in range( seleccImag_cant ):

      if len(clasesListDisp)==0:
        print("-- no hay más imágenes disponibles para seleccionar.")
        break

      # selecciona al azar una imagen de la lista
      auxiSelCl = random.choice(clasesListDisp)
      if (auxiSelCl not in clasesFN) or len(clasesFN[auxiSelCl])==0:
        print("-- no hay más imágenes disponibles para seleccionar de la clase ", auxiSelCl)
        break
      auxiSelFN = random.choice(clasesFN[auxiSelCl])

      # carga la imagen seleccionada al azar
      auxiSelImg = ImPIL.open(auxiSelFN)

      # toma el tamaño de la imagen
      auxSelImg_ancho, auxSelImg_alto  = auxiSelImg.size

      # si corresponde ajustar tamaño de las imágenes
      if ajustarTamanoImagenes and\
       (porcAjusteTamImagenMin!=1.0) and\
        (porcAjusteTamImagenMax!=1.0):
          # determina nuevo tamaño
          porc = random.uniform(porcAjusteTamImagenMin, porcAjusteTamImagenMax)
          auxSelImg_ancho = int(auxSelImg_ancho*porc)
          if (not aplicarMismoAjusteAltoAncho):
            porc = random.uniform(porcAjusteTamImagenMin, porcAjusteTamImagenMax)
          auxSelImg_alto = int(auxSelImg_alto*porc)
          # controla nuevo tamaño
          auxSelImg_ancho = max(auxSelImg_ancho, 10)
          auxSelImg_alto = max(auxSelImg_alto, 10)
          # realiza el ajuste del tamaño
          auxiSelImg = auxiSelImg.resize((auxSelImg_ancho, auxSelImg_alto), resample=ImPIL.LANCZOS)

      # determina posición de la imagen a ubicar

      # si no se mueve en el eje X o no entra más en el eje X
      auxDist = determinaDistanciaImagenes(distanciaImagenesGeneradasX, auxSelImg_ancho)
      if (not imGeneradasDespX) or ( (ultimaUbicacionX2 + auxDist + auxSelImg_ancho) >= nuevaImgAnchoTotal ):
          noPuedeAgregar = True
          if (imGeneradasDespY):
            # si se mueve en el eje Y, ve si entraría otra fila
            auxDist = determinaDistanciaImagenes(distanciaImagenesGeneradasY, auxSelImg_alto)
            if ( (maximaUbicacionY + auxDist + auxSelImg_alto) <  nuevaImgAltoTotal ):
              # lo agrega
              minimaUbicacionY = maximaUbicacionY + auxDist//3
              ultimaUbicacionY1 = minimaUbicacionY + 2*auxDist//3
              ultimaUbicacionX1 = determinaDistanciaImagenes(distanciaImagenesGeneradasX, auxSelImg_ancho)
              noPuedeAgregar = False
          if noPuedeAgregar:
            # si no puede agregar, finaliza
            print("-- no se agregan más imágenes.")
            break
      else:
          # desplaza en el eje X
          ultimaUbicacionX1 = ultimaUbicacionX2 + auxDist
          if (imGeneradasDespY):
            # si también se mueve en el eje Y, mueve también un poco (si hay lugar)
            auxDist = determinaDistanciaImagenes(distanciaImagenesGeneradasY, auxSelImg_alto)
            if ( (ultimaUbicacionY1 + auxDist + auxSelImg_alto) < nuevaImgAltoTotal ):
              ultimaUbicacionY1 = minimaUbicacionY + auxDist

      # detmina posiciones máximas
      ultimaUbicacionX2 = ultimaUbicacionX1 + auxSelImg_ancho
      ultimaUbicacionY2 = ultimaUbicacionY1 + auxSelImg_alto

      # actualiza máximos usados
      maximaUbicacionX = max(maximaUbicacionX, ultimaUbicacionX2)
      minimaUbicacionY = min(minimaUbicacionY, ultimaUbicacionY1)
      maximaUbicacionY = max(maximaUbicacionY, ultimaUbicacionY2)

      # pega la imagen seleccionada
      nuevaImg.paste(auxiSelImg, (ultimaUbicacionX1, ultimaUbicacionY1))

      # define las coordenadas para el XML con la clase correspondinete
      xml_annotation_list.append( [ (ultimaUbicacionX1 + XMLajustePosicionesX),
                                    (ultimaUbicacionY1 + XMLajustePosicionesY),
                                    (ultimaUbicacionX2 - XMLajustePosicionesX),
                                    (ultimaUbicacionY2 - XMLajustePosicionesY),
                                     auxiSelCl ] )

      # elimina seleccionada (si corresponde)
      if eliminar_elegidas:
        clasesFN[auxiSelCl].remove(auxiSelFN)
        if len(clasesFN[auxiSelCl])==0:
          del clasesFN[auxiSelCl]
          clasesListDisp.remove(auxiSelCl)

      # concatena la letra al nombre  a la lista
      nombreImagComb = nombreImagComb + auxiSelCl
      if auxiSelCl not in list_clases:
        list_clases.append(auxiSelCl)

  # define nombre de archivos
  nombreImagCombFN = nombreImagComb + '.png'
  xmlnombreImagCombFN = nombreImagComb + '.xml'

  # ajusta el tamaño de la imagen de acuerdo a lo usado
  # (así saca todo el espacio blanco que no se usa)
  maximaUbicacionX += 5
  maximaUbicacionY += 5
  nuevaImg = nuevaImg.crop((0, 0, maximaUbicacionX, maximaUbicacionY))

  # graba la nueva imagen
  nuevaImg.save( pathDestino + "/" + nombreImagCombFN, "PNG")

  # genera el XML de la nueva imagen
  xml_annotation = ET.Element('annotation')
  ET.SubElement(xml_annotation, 'folder').text = pathDestino
  ET.SubElement(xml_annotation, 'filename').text = nombreImagCombFN
  ET.SubElement(xml_annotation, 'path').text = pathDestino + "/" + nombreImagCombFN

  xml_source = ET.SubElement(xml_annotation, 'source')
  ET.SubElement(xml_source, 'database').text = 'Unknown'

  xml_size = ET.SubElement(xml_annotation, 'size')
  ET.SubElement(xml_size, 'width').text = str(maximaUbicacionX)
  ET.SubElement(xml_size, 'height').text = str(maximaUbicacionY)
  ET.SubElement(xml_size, 'depth').text = str(3)

  ET.SubElement(xml_annotation, 'segmented').text = '0'

  for annot in xml_annotation_list:
    #tmp_annot = annot.split(',')
    xmin, ymin, xmax, ymax, label = annot[0], annot[1], annot[2], annot[3], annot[4]

    object = ET.SubElement(xml_annotation, 'object')
    ET.SubElement(object, 'name').text = label
    ET.SubElement(object, 'pose').text = 'Unspecified'
    ET.SubElement(object, 'truncated').text = '0'
    ET.SubElement(object, 'difficult').text = '0'

    xml_bndbox = ET.SubElement(object, 'bndbox')
    ET.SubElement(xml_bndbox, 'xmin').text = str(xmin)
    ET.SubElement(xml_bndbox, 'ymin').text = str(ymin)
    ET.SubElement(xml_bndbox, 'xmax').text = str(xmax)
    ET.SubElement(xml_bndbox, 'ymax').text = str(ymax)

  xml_tree = ET.ElementTree(xml_annotation)
  xml_tree.write( pathDestino + "/" + xmlnombreImagCombFN )

  # muestra cada tanto para que haga más rápido que si muestra todas
  if (i%auxiCantMostrarProgreso)==0:
    print('\n(', i+1, ')- IMAGEN GENERADA: ', nombreImagComb, '(', nombreImagCombFN, ' + ', xmlnombreImagCombFN, ')' )
  else:
    print('.', end='')

print('\n** Proceso de generación de imágenes terminado **\n')
print('\n> Clases usadas para generar imágenes: ', list_clases)




** Comienza proceso de generación de imágenes **


( 1 )- IMAGEN GENERADA:  cartel_184597259816274 ( cartel_184597259816274.png  +  cartel_184597259816274.xml )
.......................................................................................................................................................................................................
( 201 )- IMAGEN GENERADA:  cartel_509397807401189 ( cartel_509397807401189.png  +  cartel_509397807401189.xml )
.......................................................................................................................................................................................................
( 401 )- IMAGEN GENERADA:  cartel_054847256749837 ( cartel_054847256749837.png  +  cartel_054847256749837.xml )
.......................................................................................................................................................................................................
( 601 )- IMAGE

In [8]:
#@title Fuerza la actualizacion del Drive

# actualiza y desmonta
drive.flush_and_unmount()

# vuelve a montar
drive.mount('/content/gdrive', force_remount=True)

print("Actualización terminada.")

Mounted at /content/gdrive
Actualización terminada.
